In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import numpy as np
from acquire import remove_stopwords, basic_clean, tokenize 
from prepare_jag import prep_train, basic_clean3
import re
import os
from re import search
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import keras

from nltk.corpus import stopwords
import nltk

In [2]:
df = prep_train()

Test, notes, and features loaded.
Merged dataframes
Renamed 'pn_history' column to 'original'
Added a basic clean column lowercaseing and removing special characters
Added stemmed column with tokenized words and stopwords removed
Added lemmatized column with lemmatized words and stopwords removed
Data preparation complete


In [3]:
df

,id,case_num,pn_num,feature_num,feature_text,annotation,location,original,clean,stemmed,lemmatized
0,00016_000,0,16,0,Family-history-of-MI-OR-Family-history-of-myoc...,['dad with recent heart attcak'],['696 724'],HPI: 17yo M presents with palpitations. Patien...,hpi 17yo presents palpitations patient reports...,hpi 17yo present palpit patient report 3-4 mon...,hpi 17yo present palpitation patient report 3-...
1,00041_000,0,41,0,Family-history-of-MI-OR-Family-history-of-myoc...,[],[],17 Y/O M CAME TO THE CLINIC C/O HEART POUNDING...,17 y/o came clinic c/o heart pounding started ...,17 y/o came clinic c/o heart pound start 2-3 m...,17 y/o came clinic c/o heart pounding started ...
2,00046_000,0,46,0,Family-history-of-MI-OR-Family-history-of-myoc...,['father: heart attack'],['824 844'],Mr. Cleveland is a 17yo M who was consented by...,mr cleveland 17yo consented parents alone toda...,mr cleveland 17yo wa consent parent alon today...,mr cleveland 17yo wa consented parent alone to...
3,00082_000,0,82,0,Family-history-of-MI-OR-Family-history-of-myoc...,['Father MI'],['622 631'],17 yo M w/ no cardiac or arrhythmia PMH presen...,17 yo w/ no cardiac arrhythmia pmh presents co...,17 yo w/ no cardiac arrhythmia pmh present com...,17 yo w/ no cardiac arrhythmia pmh present com...
4,00100_000,0,100,0,Family-history-of-MI-OR-Family-history-of-myoc...,['Dad-MI'],['735 741'],HPI: Dillon Cleveland is an otherwise healthy ...,hpi dillon cleveland otherwise healthy 17 year...,hpi dillon cleveland otherwis healthi 17 year ...,hpi dillon cleveland otherwise healthy 17 year...
...,...,...,...,...,...,...,...,...,...,...,...
14295,95145_916,9,95145,916,Subjective-fever,['subjective fever'],['169 185'],Pt is 20 yo F w headache since yesterday morni...,pt 20 yo f w headache since yesterday morning ...,pt 20 yo f w headach sinc yesterday morn pt st...,pt 20 yo f w headache since yesterday morning ...
14296,95228_916,9,95228,916,Subjective-fever,[],[],"20 F no PMH, lives w/ roommate in apartment ha...",20 f no pmh lives w/ roommate apartment severe...,20 f no pmh live w/ roommat apart ha sever hea...,20 f no pmh life w/ roommate apartment ha seve...
14297,95243_916,9,95243,916,Subjective-fever,['feels warm'],['376 386'],20 y/o F with no PMH is presenting with 1 day ...,20 y/o f no pmh presenting 1 day history heada...,20 y/o f no pmh present 1 day histori headach ...,20 y/o f no pmh presenting 1 day history heada...
14298,95330_916,9,95330,916,Subjective-fever,['Felt warm'],['358 367'],Ms. Madden is a 20 yo female presenting w/ the...,ms madden 20 yo female presenting w/ worst ha ...,ms madden 20 yo femal present w/ worst ha life...,madden 20 yo female presenting w/ worst ha lif...


In [4]:
df.shape

(14300, 11)

In [5]:
df.clean[0]

'hpi 17yo presents palpitations patient reports 3-4 months intermittent episodes heart beating/pounding chest 2 days ago soccer game episode time chest pressure felt going pass lose conciousness note patient endorses abusing adderall primarily study 1-3 times per week recent soccer game took adderrall night morning game denies shortness breath diaphoresis fevers chills headache fatigue changes sleep changes vision/hearing abdominal paun changes bowel urinary habits pmhx none rx uses friends adderrall fhx mom thyroid disease dad recent heart attcak none immunizations date shx freshmen college endorses 3-4 drinks 3 nights / week weekends denies tabacco endorses trying marijuana sexually active girlfriend x 1 year uses condoms'

In [6]:
df.feature_text[0]

'Family-history-of-MI-OR-Family-history-of-myocardial-infarction'

## Multi-label binarization
Let's preprocess our labels using the [StringLookup](https://keras.io/api/layers/preprocessing_layers/categorical/string_lookup) layer.

In [7]:
features = tf.ragged.constant(df['feature_text'].values)
lookup = tf.keras.layers.StringLookup(output_mode='multi_hot')
lookup.adapt(features)
vocab = lookup.get_vocabulary()

2022-02-24 08:10:37.568402: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
vocab

['[UNK]',
 'Female',
 'Male',
 'Nausea',
 '35-year',
 '20-year',
 '17-year',
 'viral-symptoms-OR-rhinorrhea-OR-scratchy-throat',
 'tossing-and-turning',
 'symptoms-for-6-months',
 'loss-of-interest',
 'heavy-periods-OR-irregular-periods',
 'heart-pounding-OR-heart-racing',
 'getting-worse-OR-progressive-OR-symptoms-now-daily',
 'duration-3-weeks',
 'duration-2-months',
 'burning-OR-gnawing-OR-burning-and-gnawing',
 'anxious-OR-nervous',
 'Worse-with-deep-breath-OR-pleuritic',
 'Weight-stable',
 'Weight-loss',
 'Weight-Gain',
 'Vomiting',
 'Visual-hallucination-once',
 'Vaginal-dryness',
 'Unsuccessful-napping',
 'Unprotected-Sex',
 'Subjective-fevers',
 'Subjective-fever',
 'Stress-due-to-caring-for-elderly-parents',
 'Stress',
 'Son-died-3-weeks-ago',
 'Sleeping-medication-ineffective',
 'Sleep-disturbance-OR-Early-awakenings',
 'Shortness-of-breath',
 'Sharp-OR-stabbing-OR-7-to-8-out-of-10-on-pain-scale',
 'Shares-an-apartment',
 'Sexually-active',
 'Right-sided-LQ-abdominal-pain-OR-

In [9]:
def invert_multi_hot(encoded_labels):
    '''Reverse a single multi-hot encoded label to a tuple of vocab terms.'''
    hot_indices = np.argwhere(encoded_labels == 1.0)[...,0]
    return np.take(vocab, hot_indices)

print('Vocabulary:\n')
print(vocab)

Vocabulary:

['[UNK]', 'Female', 'Male', 'Nausea', '35-year', '20-year', '17-year', 'viral-symptoms-OR-rhinorrhea-OR-scratchy-throat', 'tossing-and-turning', 'symptoms-for-6-months', 'loss-of-interest', 'heavy-periods-OR-irregular-periods', 'heart-pounding-OR-heart-racing', 'getting-worse-OR-progressive-OR-symptoms-now-daily', 'duration-3-weeks', 'duration-2-months', 'burning-OR-gnawing-OR-burning-and-gnawing', 'anxious-OR-nervous', 'Worse-with-deep-breath-OR-pleuritic', 'Weight-stable', 'Weight-loss', 'Weight-Gain', 'Vomiting', 'Visual-hallucination-once', 'Vaginal-dryness', 'Unsuccessful-napping', 'Unprotected-Sex', 'Subjective-fevers', 'Subjective-fever', 'Stress-due-to-caring-for-elderly-parents', 'Stress', 'Son-died-3-weeks-ago', 'Sleeping-medication-ineffective', 'Sleep-disturbance-OR-Early-awakenings', 'Shortness-of-breath', 'Sharp-OR-stabbing-OR-7-to-8-out-of-10-on-pain-scale', 'Shares-an-apartment', 'Sexually-active', 'Right-sided-LQ-abdominal-pain-OR-Right-lower-quadrant-abdo

### Separate the individual targets from the label pool and then use it to represent a given label set with 0's and 1's

In [11]:
sample_label = df['feature_text'].iloc[0]
print(f'Original label: {sample_label}')

label_binarized = lookup([sample_label])
print(f'Label-binarized representation: {label_binarized}')

Original label: Family-history-of-MI-OR-Family-history-of-myocardial-infarction
Label-binarized representation: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Data preprocessing and [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) objects

In [12]:
df['clean'].apply(lambda x: len(x.split(" "))).describe()

count    14300.000000
mean        96.436853
std         16.015234
min         26.000000
25%         87.000000
50%         98.000000
75%        108.000000
max        134.000000
Name: clean, dtype: float64

# Takeaways
* Half of the student notes have a length of 97 words.

In [13]:
max_seqlen = 98
batch_size = 128
padding_token = '<pad>'
auto = tf.data.AUTOTUNE

def make_dataset(dataframe, is_train=True):
    labels = tf.ragged.constant(dataframe['feature_text'].values)
    label_binarized = lookup(labels).numpy()
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe['clean'].values, label_binarized)
    )
    dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
    return dataset.batch(batch_size)

## Prepare the [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) objects.

In [19]:
len(df.feature_text)

14300

In [21]:
len(df.clean)

14300

In [22]:
train_dataset = make_dataset(df, is_train=False)

ValueError: Dimensions 14300 and 132 are not compatible

## Preview the dataset

In [ ]:
text_batch, label_batch = next(iter(train_dataset))

for i, text in enumerate(text_batch[:5]):
    label = label_batch[i].numpy()[None, ...]
    print(f'Student note: {text}')
    print(f'Targets: {invert_multi_hot(label[0])}')
    print(' ')

## Vectorization
Vectorize the text to represent it as a quantitative value. We will use [TextVectorization layer](https://keras.io/api/layers/preprocessing_layers/text/text_vectorization)

In [ ]:
# Get unique words in student notes.
vocabulary = set()
train_df['clean'].str.split().apply(vocabulary.update)
vocabulary_size = len(vocabulary)
print(vocabulary_size)

## Now we create our vectorization layer and map() to the [tf.data.Datasets](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) created earlier.

In [ ]:
text_vectorizer = layers.TextVectorization(
    max_tokens=vocabulary_size, ngrams=2, output_mode='tf_idf'
)

with tf.device('/CPU:0'):
    text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

train_dataset = train_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto).prefetch(auto)
validation_dataset = validation_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto).prefetch(auto)
test_dataset = test_dataset.map(
    lambda test, label: (text_vectorizer(text), label), num_parallel_calls=auto).prefetch(auto)

## Create a text classification model

In [ ]:
def make_model():
    shallow_mlp_model = keras.Sequential(
        [
            layers.Dense(512, activation="relu"),
            layers.Dense(256, activation="relu"),
            layers.Dense(lookup.vocabulary_size(), activation="softmax"),
        ]  # More on why "sigmoid" has been used here in a moment.
    )
    return shallow_mlp_model

## Train our model

In [ ]:
epochs = 10

shallow_mlp_model = make_model()
shallow_mlp_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"]
)

history = shallow_mlp_model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs
)


def plot_result(item):
    plt.plot(history.history[item], label=item)
    plt.plot(history.history["val_" + item], label="val_" + item)
    plt.xlabel("Epochs")
    plt.ylabel(item)
    plt.title("Train and Validation {} Over Epochs".format(item), fontsize=14)
    plt.legend()
    plt.grid()
    plt.show()


plot_result("loss")
plot_result("categorical_accuracy")

## Evaluate the model

In [ ]:
_, categorical_acc = shallow_mlp_model.evaluate(test_dataset)
print(f"Categorical accuracy on the test set: {round(categorical_acc * 100, 2)}%.")

In [ ]:
# Create a model for inference.
model_for_inference = keras.Sequential([text_vectorizer, shallow_mlp_model])

# Create a small dataset just for demoing inference.
inference_dataset = make_dataset(test_df.sample(100), is_train=False)
text_batch, label_batch = next(iter(inference_dataset))
predicted_probabilities = model_for_inference.predict(text_batch)

# Perform inference.
for i, text in enumerate(text_batch[10:50]):
    label = label_batch[i].numpy()[None, ...]
    print(f"Student notes: {text}")
    print(f"Targets(s): {invert_multi_hot(label[0])}")
    predicted_proba = [proba for proba in predicted_probabilities[i]]
    all_labels = [
        x
        for _, x in sorted(
            zip(predicted_probabilities[i], lookup.get_vocabulary()),
            key=lambda pair: pair[0],
            reverse=True,
            
        )
    ][:]
    print(f"Predicted Targets(s): ({', '.join([label for label in all_labels])})")
    print(" ")